### Lyrics Downloader

Use MusixMatch API to download and populate lyrics into the dataset.

In [ ]:
import pandas as pd
import numpy as np

from lyrics_service import lyrics_finder

In [2]:
# skiprows used to ignore already downloaded lyrics
data = pd.read_csv('../data/test.csv') # ,skiprows=[i for i in range(1, 385)])
data.head()

,dzr_sng_id,MSD_sng_id,MSD_track_id,valence,arousal,artist_name,track_name
0,532171,SOSDIDR12A8C130E5D,TRGVJCD128F42623E3,-0.420759,0.754644,Fiona Apple,Fast As You Can
1,532216,SOPMTTO12AB0183DB9,TRNXMAY128F931FAC6,-0.465330,0.011006,Fiona Apple,I Know
2,532284,SOQCEJU12A8C13220B,TRYMZQJ128F426A0E7,0.373325,-0.923151,Xzibit,Get Your Walk On
3,532989,SOLEFHR12D021B2A57,TRZHHIL128F4264EE0,0.894528,-0.390774,Fiona Apple,Extraordinary Machine
4,533164,SOFDPOX12A8C13780D,TRJEBIT128F933A400,-1.636900,-0.459145,Patty Loveless,How Can I Help You Say Goodbye


In [4]:
track_name = data["track_name"]
artist_name = data["artist_name"]
track_name.shape, artist_name.shape

((3514,), (3514,))

In [5]:
# list of downloaded lyrics
lyrics_list = []

for i in range(len(track_name)):
    status_code, lyrics = lyrics_finder("jsonp", "callback", track_name.loc[i], artist_name.loc[i],
                                         "37d7829ae9c5bf71588c67bccf16cdde")
    
    if status_code == 200:
        lyrics_list.append(lyrics["lyrics"]["lyrics_body"])
    else:
        lyrics_list.append(None)
    
len(lyrics_list)

3514

In [7]:
data_copy = data.copy()

In [8]:
lyrics_series = pd.Series(None, np.arange(len(track_name)))
lyrics_series.iloc[: len(lyrics_list)] = lyrics_list

<ipython-input-8-1775ddad768f>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(None, np.arange(len(track_name)))


In [43]:
current = 385 - 1

In [44]:
data_copy["Lyrics"][current:] = lyrics_series[:]
data_copy["Lyrics"].iloc[current:]

<ipython-input-44-dc127d992fd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation["Lyrics"][current:] = s[:]


384     Go on and wring my neck\nLike when a rag gets ...
385     Drought makes the workers dream\nMuscles and f...
386     We were so happy\nThings worked out better tha...
387     I know it sounds funny but I just can't stand ...
388     Melancholy\nAviation\nChocolate\nPerfume\nCiga...
                              ...                        
3858    On the fields of desolation\nWe are all alone\...
3859    Is it a sin, to love again?\nI cannot win, thi...
3860    Though you and I would dance and laugh and pla...
3861    Hello?\nIs there anybody in there?\nJust nod i...
3862    Money\nGet away\nYou get a good job with good ...
Name: Lyrics, Length: 3479, dtype: object

In [10]:
data_copy.to_csv("../data/test.csv", index=False)